# Customer Service Discord Bot with Agentic RAG

You can also check this cookbook in colab [here]()

⭐ **Star the Repo**

If you find CAMEL useful or interesting, please consider giving it a star on our [CAMEL GitHub Repo](https://github.com/camel-ai/camel)! Your stars help others find this project and motivate us to continue improving it.

########Picture

## Installation and Setup
First, install the CAMEL package with all its dependencies

In [ ]:
!pip install "camel-ai[all]==0.2.12"
!pip install nest_asyncio

Next, set up your API keys for Firecrawl and the model (Qwen or Mistral)

If you don't have a FireCrawl API key, you can obtain one by following these steps:

1. Visit the FireCrawl API Key page https://www.firecrawl.dev/app/api-keys

2. Log in or sign up for a FireCrawl account.

3. Navigate to the 'API Keys' section.

4. Click on 'Create API Key' button to generate a new API key.

For more details, you can also check the Firecrawl documentation: https://docs.firecrawl.dev/api-reference/introduction

In [7]:
import os
from getpass import getpass

firecrawl_api_key = getpass('Enter your API key: ')
os.environ["FIRECRAWL_API_KEY"] = firecrawl_api_key

If you want to choose Mistral as the model, skip below part for Qwen.

If you don't have a Qwen API key, you can obtain one by following these steps:

1. Visit the Alibaba Cloud Model Studio Console (https://www.alibabacloud.com/en?_p_lc=1) and follow the on-screen instructions to activate the model services.

2. In the upper-right corner of the console, click on your account name and select API-KEY.

3. On the API Key management page, click on the Create API Key button to generate a new key.

For more details, you can also check the Qwen documentation: https://www.alibabacloud.com/help/en/model-studio/developer-reference/use-qwen-by-calling-api

In [8]:
import os
from getpass import getpass

qwen_api_key = getpass('Enter your API key: ')
os.environ["QWEN_API_KEY"] = qwen_api_key

Alternatively, use Mistral.

If you don't have a Mistral API key, you can obtain one by following these steps:

1. Visit the Mistral Console (https://console.mistral.ai/)

2. In the left panel, click on API keys under API section

3. Choose your plan

For more details, you can also check the Mistral documentation: https://docs.mistral.ai/getting-started/quickstart/

In [9]:
import os
from getpass import getpass

mistral_api_key = getpass('Enter your API key')
os.environ["MISTRAL_API_KEY"] = mistral_api_key

## Knowledge Crawling and Storage

Use Firecrawl to crawl a website and store the content in a markdown file:

In [10]:
import os
from camel.loaders import Firecrawl

os.makedirs('local_data', exist_ok=True)

firecrawl = Firecrawl()

knowledge = firecrawl.crawl(
    url="https://qdrant.tech/documentation/overview/"
)["data"][0]["markdown"]

with open('local_data/qdrant_overview.md', 'w') as file:
    file.write(knowledge)

## Basic Agent Setup

Qwen is a large language model developed by Alibaba Cloud. It is trained on a massive dataset of text and code and can generate text, translate languages, write different kinds of creative content, and answer your questions in an informative way.

Use QWen model:

In [11]:
from camel.configs import QwenConfig, MistralConfig
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType

qwen_model = ModelFactory.create(
    model_platform=ModelPlatformType.QWEN,
    model_type=ModelType.QWEN_TURBO,
    model_config_dict=QwenConfig(temperature=0.2).as_dict(),
)

mistral_model = ModelFactory.create(
    model_platform=ModelPlatformType.MISTRAL,
    model_type=ModelType.MISTRAL_LARGE,
    model_config_dict=MistralConfig(temperature=0.0).as_dict(),
)

# Use Qwen model
model = qwen_model

# Replace with mistral_model if you want to choose mistral mode instead
# model = mistral_model

In [12]:
from camel.agents import ChatAgent
from camel.messages import BaseMessage

agent = ChatAgent(
    system_message="You're a helpful assistant",
    message_window_size=10,
    model=model
)

knowledge_message = BaseMessage.make_user_message(
    role_name="User", content=f"Based on the following knowledge: {knowledge}"
)
agent.update_memory(knowledge_message, "user")

## Basic Chatbot Setup

In [ ]:
print("Start chatting! Type 'exit' to end the conversation.")
while True:
    user_input = input("User: ")
    print(f"User: {user_input}")

    if user_input.lower() == "exit":
        print("Ending conversation.")
        break

    assistant_response = agent.step(user_input)
    print(f"Assistant: {assistant_response.msgs[0].content}")

Start chatting! Type 'exit' to end the conversation.
2024-12-16 17:10:14,437 - httpx - INFO - HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
2024-12-16 17:10:14,447 - camel.agents.chat_agent - INFO - Model qwen-turbo, index 0, processed these messages: [{'role': 'system', 'content': "You're a helpful assistant"}, {'role': 'user', 'content': 'Based on the following knowledge: ## Docs Menu\n\n- [Documentation](https://qdrant.tech/documentation/)\n- [Overview](https://qdrant.tech/documentation/overview/)\n- Understanding Vector Search in Qdrant\n\n# [Anchor](https://qdrant.tech/documentation/overview/vector-search/\\#how-does-vector-search-work-in-qdrant) How Does Vector Search Work in Qdrant?\n\nHow Vector Search Algorithms Work: An Intro to Qdrant - YouTube\n\nQdrant - Vector Database & Search Engine\n\n6.22K subscribers\n\n[How Vector Search Algorithms Work: An Intro to Qdrant](https://www.youtube.com/watch?v=mXNrhyw4q84)\n\nQdra

## Basic Slack Bot Integration

To build a slack bot, you need to access these following informations.
1. Go to the slack app website: https://api.slack.com/apps
2. Log in with your slack account, or create an account if you don't have one
3. Click on 'Create New App' to create a new bot

In [13]:
import os
from getpass import getpass
slack_token = getpass('Enter your slack token: ')
slack_scopes = getpass('Enter your slack scopes: ')
slack_signing_secret = getpass('Enter your slack signing secret')
slack_client_id = getpass('Enter your slack client id')
slack_client_secret = getpass('Enter your slack client secret')

os.environ['SLACK_TOKEN'] = slack_token
os.environ['SLACK_SCOPES'] = slack_scopes
os.environ['SLACK_SIGNING_SECRET'] = slack_signing_secret
os.environ['SLACK_CLIENT_ID'] = slack_client_id
os.environ['SLACK_CLIENT_SECRET'] = slack_client_secret

In [14]:
from camel.bots import SlackApp

slack_bot = SlackApp(
    token=slack_token,
    scopes=slack_scopes,
    signing_secret=slack_signing_secret,
    client_id=slack_client_id,
    client_secret=slack_client_secret
)

2024-12-16 17:38:55,499 - camel.bots.slack.slack_app - WARNING - As `installation_store` or `authorize` has been used, `token` (or SLACK_BOT_TOKEN env variable) will be ignored.


Rewrite on_message method to custom message processing flow

In [16]:
import nest_asyncio
import types

nest_asyncio.apply()

# async def custom_on_message(self, event, say):
#     print(f"User: {event}")

# slack_bot.on_message = types.MethodType(custom_on_message, slack_bot)
slack_bot.run()

2024-12-16 17:43:45,490 - camel.bots.slack.slack_app - INFO - ⚡️ Bolt app is running!


RuntimeError: web.Application instance initialized with different loop

## Integrating Qdrant for Large Files to build a more powerful Discord bot

Qdrant is a vector similarity search engine and vector database. It is designed to perform fast and efficient similarity searches on large datasets of vectors. This enables the chatbot to access and utilize external information to provide more comprehensive and accurate responses. By storing knowledge as vectors, Qdrant enables efficient semantic search, allowing the chatbot to find relevant information based on the meaning of the user's query.

Set up an embedding model and retriever for Qdrant:

In [ ]:
from camel.embeddings import SentenceTransformerEncoder

sentence_encoder = SentenceTransformerEncoder(model_name='intfloat/e5-large-v2')

In [ ]:
from camel.retrievers import AutoRetriever
from camel.types import StorageType

assistant_sys_msg = """You are a helpful assistant to answer question,
         I will give you the Original Query and Retrieved Context,
        answer the Original Query based on the Retrieved Context,
        if you can't answer the question just say I don't know."""
auto_retriever = AutoRetriever(
              vector_storage_local_path="local_data2/",
              storage_type=StorageType.QDRANT,
              embedding_model=sentence_encoder
            )
qdrant_agent = ChatAgent(system_message=assistant_sys_msg, model=model)

Use Auto RAG to retrieve first and then answer the user's query using CAMEL `ChatAgent` based on the retrieved info:

In [ ]:
from camel.bots import DiscordApp
import nest_asyncio
import discord

nest_asyncio.apply()
discord_q_bot = DiscordApp(token=discord_bot_token)

@discord_q_bot.client.event # triggers when a message is sent in the channel
async def on_message(message: discord.Message):
    if message.author == discord_q_bot.client.user:
        return

    if message.type != discord.MessageType.default:
        return

    if message.author.bot:
        return
    user_input = message.content

    retrieved_info = auto_retriever.run_vector_retriever(
        query=user_input,
        contents=[
            "local_data/qdrant_overview.md",
        ],
        top_k=3,
        return_detailed_info=False,
        similarity_threshold=0.5
    )

    user_msg = str(retrieved_info)
    assistant_response = qdrant_agent.step(user_msg)
    response_content = assistant_response.msgs[0].content

    if len(response_content) > 2000: # discord message length limit
        for chunk in [response_content[i:i+2000] for i in range(0, len(response_content), 2000)]:
            await message.channel.send(chunk)
    else:
        await message.channel.send(response_content)

discord_q_bot.run()